In [ ]:
# train.py - Training and model saving component
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
import joblib
from google.colab import files

def train_model():
    # Upload your dataset
    print("Please upload your music dataset (CSV file) for training:")
    uploaded = files.upload()
    filename = next(iter(uploaded)) if uploaded else None

    try:
        if not filename:
            raise ValueError("No file uploaded")

        df = pd.read_csv(filename)
        print("\nDataset loaded successfully!")
        print(f"Found {len(df)} songs")

        # Validate required columns
        required_cols = {'labels', 'uri', 'danceability', 'energy', 'valence', 'tempo'}
        missing_cols = required_cols - set(df.columns)
        if missing_cols:
            raise ValueError(f"Missing required columns: {missing_cols}")

        # Convert labels to string if they're numeric
        if not pd.api.types.is_string_dtype(df['labels']):
            print("\nConverting numeric labels to strings...")
            df['labels'] = df['labels'].astype(str)

        print("\nDataset validation complete. Proceeding with training...")

        # Prepare KNN recommender
        features = ["danceability", "energy", "valence", "tempo"]
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(df[features])

        # Set dynamic n_neighbors based on dataset size
        n_neighbors = min(5, len(df))
        knn = NearestNeighbors(n_neighbors=n_neighbors, metric='cosine')
        knn.fit(X_scaled)

        # Save the trained models and data
        joblib.dump(scaler, 'scaler.joblib')
        joblib.dump(knn, 'knn_model.joblib')
        df.to_csv('processed_data.csv', index=False)

        print("\nTraining completed successfully!")
        print("Saved files:")
        print("- scaler.joblib: Feature scaler")
        print("- knn_model.joblib: Trained KNN model")
        print("- processed_data.csv: Processed dataset")

        return True

    except Exception as e:
        print(f"\nError during training: {str(e)}")
        return False

if __name__ == "__main__":
    train_model()

Please upload your music dataset (CSV file) for training:


Saving 278k_labelled_uri1.csv to 278k_labelled_uri1.csv

Dataset loaded successfully!
Found 4000 songs

Converting numeric labels to strings...

Dataset validation complete. Proceeding with training...

Training completed successfully!
Saved files:
- scaler.joblib: Feature scaler
- knn_model.joblib: Trained KNN model
- processed_data.csv: Processed dataset


In [ ]:
# # predict.py - Prediction and recommendation component
# import pandas as pd
# import joblib
# from flask_ngrok import run_with_ngrok
# from flask import Flask, jsonify
# import os

# # Initialize Flask app
# app = Flask(__name__)
# run_with_ngrok(app)

# # Load trained models and data
# def load_models():
#     try:
#         scaler = joblib.load('scaler.joblib')
#         knn = joblib.load('knn_model.joblib')
#         df = pd.read_csv('processed_data.csv')
#         return scaler, knn, df
#     except Exception as e:
#         print(f"Error loading models: {str(e)}")
#         return None, None, None

# scaler, knn, df = load_models()
# features = ["danceability", "energy", "valence", "tempo"]

# def get_recommendations(emotion):
#     """Get recommendations using the trained model"""
#     if scaler is None or knn is None or df is None:
#         return {"error": "Models not loaded properly"}

#     emotion_str = str(emotion).lower()
#     emotion_songs = df[df['labels'].str.lower() == emotion_str]

#     if len(emotion_songs) == 0:
#         return {"error": f"No songs found for emotion: {emotion}"}

#     # KNN-based recommendations
#     avg_features = emotion_songs[features].mean().values.reshape(1, -1)
#     avg_features_scaled = scaler.transform(avg_features)
#     _, indices = knn.kneighbors(avg_features_scaled)

#     # Convert URIs to playable URLs
#     recommended = []
#     for uri in df.iloc[indices[0]]['uri']:
#         if uri.startswith('spotify:track:'):
#             recommended.append(f"https://open.spotify.com/track/{uri.split(':')[-1]}")
#         else:
#             recommended.append(uri)
#     return recommended

# @app.route('/recommend/<emotion>')
# def recommend(emotion):
#     recommended_songs = get_recommendations(emotion)

#     if isinstance(recommended_songs, dict) and 'error' in recommended_songs:
#         return jsonify(recommended_songs), 400

#     return jsonify({
#         'predicted_emotion': emotion,
#         'recommended_songs': recommended_songs,
#         'dataset_info': {
#             'total_songs': len(df),
#             f"songs_for_{emotion}": len(df[df['labels'].str.lower() == str(emotion).lower()])
#         }
#     })

# if __name__ == '__main__':
#     if scaler and knn and df is not None:
#         print("Model loaded successfully. Starting server...")
#         app.run()
#     else:
#         print("Failed to load models. Please train the model first.")

In [ ]:
"""
from flask import Flask, request, jsonify
from flask_cors import CORS
import torch
import torch.nn as nn
import torch.nn.functional as F
import cv2
import numpy as np
import io
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import random
from pyngrok import ngrok
import joblib
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler

# Initialize Flask app
app = Flask(__name__)
CORS(app, resources={r"/*": {"origins": "*"}})

# Spotify API credentials
SPOTIFY_CLIENT_ID = '6d07d59d7bd6434e93d9522ed98932a5'
SPOTIFY_CLIENT_SECRET = '9f948e31b5304c27b6fb5661b95c45b3'
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    client_id=SPOTIFY_CLIENT_ID,
    client_secret=SPOTIFY_CLIENT_SECRET
))

# Set ngrok auth token
ngrok.set_auth_token("2rcQPgulcm6DfbhT777NQESmLEC_aewARhgxDRvRDnLSpV1Z")

# ====================== Emotion CNN Setup ======================
class EmotionCNN(nn.Module):
    def __init__(self, num_classes):
        super(EmotionCNN, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.fc1 = nn.Linear(128 * 6 * 6, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Load emotion model
model = EmotionCNN(num_classes=3)
model.load_state_dict(torch.load('emotion_cnn.pth'))
model.eval()

# Emotion classes
class_indices = {'happy': 0, 'neutral': 1, 'surprise': 2}
index_to_class = {v: k for k, v in class_indices.items()}

# ====================== Music Recommendation Setup ======================
# Load KNN recommendation models
try:
    scaler = joblib.load('scaler.joblib')
    knn = joblib.load('knn_model.joblib')
    df = pd.read_csv('processed_data.csv')
    features = ["danceability", "energy", "valence", "tempo"]
    print("Music recommendation models loaded successfully!")
except Exception as e:
    print(f"Error loading music models: {str(e)}")
    scaler, knn, df = None, None, None

# ====================== Helper Functions ======================
def preprocess_image(image_data):
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    file_bytes = np.frombuffer(image_data, np.uint8)
    img = cv2.imdecode(file_bytes, cv2.IMREAD_GRAYSCALE)

    # Detect face
    faces = face_cascade.detectMultiScale(img, scaleFactor=1.1, minNeighbors=5)

    if len(faces) == 0:
        raise Exception("No face detected")

    # Crop the first face found
    (x, y, w, h) = faces[0]
    face = img[y:y+h, x:x+w]

    # Resize and normalize
    face = cv2.resize(face, (48, 48))
    face = face / 255.0
    face = np.expand_dims(face, axis=0)
    face = np.expand_dims(face, axis=0)
    img_tensor = torch.tensor(face, dtype=torch.float32)

    return img_tensor

def get_knn_recommendations(emotion):
    # Get recommendations using the trained KNN model
    if scaler is None or knn is None or df is None:
        return {"error": "Music recommendation models not loaded properly"}

    try:
        # Convert emotion to string (handles both numeric and string labels)
        emotion_str = str(emotion)

        # Check if labels are numeric or strings
        if pd.api.types.is_numeric_dtype(df['labels']):
            # If labels are numeric (0,1,2), convert emotion to corresponding number
            emotion_label = int(emotion) if isinstance(emotion, (int, float)) else class_indices.get(emotion.lower(), 0)
            emotion_songs = df[df['labels'] == emotion_label]
        else:
            # If labels are strings ("happy", etc.), match case-insensitive
            emotion_songs = df[df['labels'].str.lower() == emotion_str.lower()]

        if len(emotion_songs) == 0:
            return {"error": f"No songs found for emotion: {emotion}"}

        # KNN-based recommendations
        avg_features = emotion_songs[features].mean().values.reshape(1, -1)
        avg_features_scaled = scaler.transform(avg_features)
        _, indices = knn.kneighbors(avg_features_scaled)

        # Convert URIs to playable URLs and get track info
        recommendations = []
        for uri in df.iloc[indices[0]]['uri']:
            if uri.startswith('spotify:track:'):
                track_id = uri.split(':')[-1]
                try:
                    track = sp.track(uri)
                    recommendations.append({
                        'name': track['name'],
                        'artist': track['artists'][0]['name'],
                        'link': track['external_urls']['spotify'],
                        'image': track['album']['images'][0]['url'] if track['album']['images'] else None
                    })
                except:
                    recommendations.append({
                        'link': f"https://open.spotify.com/track/{track_id}",
                        'name': f"Track {track_id}",
                        'artist': "Unknown",
                        'image': None
                    })
            else:
                recommendations.append({
                    'link': uri,
                    'name': "Custom Track",
                    'artist': "Unknown",
                    'image': None
                })
        return recommendations

    except Exception as e:
        return {"error": f"Error in recommendation: {str(e)}"}

def get_spotify_track(emotion):
    # Get random track from Spotify based on emotion
    results = sp.search(q=f'mood:{emotion}', type='track', limit=10)
    tracks = results['tracks']['items']

    if not tracks:
        # Default fallback track
        track = sp.track('spotify:track:4uLU6hMCjMI75M1A2tKUQC')
        return {
            'name': track['name'],
            'artist': track['artists'][0]['name'],
            'link': track['external_urls']['spotify'],
            'image': track['album']['images'][0]['url'] if track['album']['images'] else None
        }

    track = random.choice(tracks[:5])  # Random from top 5 tracks
    return {
        'name': track['name'],
        'artist': track['artists'][0]['name'],
        'link': track['external_urls']['spotify'],
        'image': track['album']['images'][0]['url'] if track['album']['images'] else None
    }

# ====================== API Endpoints ======================
@app.route('/predict', methods=['POST'])
def predict():
    if 'file' not in request.files:
        return jsonify({'error': 'No file uploaded'}), 400

    file = request.files['file']
    if file.filename == '':
        return jsonify({'error': 'No file selected'}), 400

    try:
        # Read and preprocess the image
        image_data = file.read()
        processed_img = preprocess_image(image_data)

        # Get emotion prediction
        prediction = model(processed_img)
        predicted_class_index = torch.argmax(prediction, dim=1).item()
        predicted_class_label = index_to_class[predicted_class_index]

        # Get both KNN and Spotify recommendations
        knn_recommendations = get_knn_recommendations(predicted_class_label)
        spotify_recommendation = get_spotify_track(predicted_class_label)

        response = {
            'predicted_class': predicted_class_label,
            'knn_recommendations': knn_recommendations if not isinstance(knn_recommendations, dict) else [],
            'spotify_recommendation': spotify_recommendation,
            'model_used': 'knn' if not (isinstance(knn_recommendations, dict) and 'error' in knn_recommendations) else 'spotify'
        }
        return jsonify(response), 200

    except Exception as e:
        return jsonify({'error': str(e)}), 500

@app.route('/recommend/<emotion>', methods=['GET'])
def recommend(emotion):
    # Direct recommendation endpoint (for testing)
    try:
        knn_recommendations = get_knn_recommendations(emotion)
        spotify_recommendation = get_spotify_track(emotion)

        response = {
            'requested_emotion': emotion,
            'knn_recommendations': knn_recommendations if not isinstance(knn_recommendations, dict) else [],
            'spotify_recommendation': spotify_recommendation,
            'model_used': 'knn' if not (isinstance(knn_recommendations, dict) and 'error' in knn_recommendations) else 'spotify'
        }
        return jsonify(response), 200
    except Exception as e:
        return jsonify({'error': str(e)}), 500

if __name__ == '__main__':
    # Start ngrok tunnel
    ngrok_tunnel = ngrok.connect(5000)
    print(' * Public URL:', ngrok_tunnel.public_url)

    # Run Flask app
    app.run(port=5000)
"""

'\nfrom flask import Flask, request, jsonify\nfrom flask_cors import CORS\nimport torch\nimport torch.nn as nn\nimport torch.nn.functional as F\nimport cv2\nimport numpy as np\nimport io\nimport spotipy\nfrom spotipy.oauth2 import SpotifyClientCredentials\nimport random\nfrom pyngrok import ngrok\nimport joblib\nimport pandas as pd\nfrom sklearn.neighbors import NearestNeighbors\nfrom sklearn.preprocessing import StandardScaler\n\n# Initialize Flask app\napp = Flask(__name__)\nCORS(app, resources={r"/*": {"origins": "*"}})\n\n# Spotify API credentials\nSPOTIFY_CLIENT_ID = \'6d07d59d7bd6434e93d9522ed98932a5\'\nSPOTIFY_CLIENT_SECRET = \'9f948e31b5304c27b6fb5661b95c45b3\'\nsp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(\n    client_id=SPOTIFY_CLIENT_ID,\n    client_secret=SPOTIFY_CLIENT_SECRET\n))\n\n# Set ngrok auth token\nngrok.set_auth_token("2rcQPgulcm6DfbhT777NQESmLEC_aewARhgxDRvRDnLSpV1Z")\n\n# ====================== Emotion CNN Setup ======================\nclass Emotio

In [ ]:
!pip install pyngrok
!pip install flask-cors
!pip install fastapi uvicorn tensorflow numpy opencv-python pillow
!pip install python-multipart
!pip install spotipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.5/261.5 kB 12.9 MB/s eta 0:00:00


In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import torch
import torch.nn as nn
import torch.nn.functional as F
import cv2
import numpy as np
import io
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import random
from pyngrok import ngrok
import joblib
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler

# Initialize Flask app
app = Flask(__name__)
CORS(app, resources={r"/*": {"origins": "*"}})

# Spotify API credentials
SPOTIFY_CLIENT_ID = '6d07d59d7bd6434e93d9522ed98932a5'
SPOTIFY_CLIENT_SECRET = '9f948e31b5304c27b6fb5661b95c45b3'
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    client_id=SPOTIFY_CLIENT_ID,
    client_secret=SPOTIFY_CLIENT_SECRET
))

# Set ngrok auth token
ngrok.set_auth_token("2rcQPgulcm6DfbhT777NQESmLEC_aewARhgxDRvRDnLSpV1Z")

# ====================== Emotion CNN Setup ======================
class EmotionCNN(nn.Module):
    def __init__(self, num_classes):
        super(EmotionCNN, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.fc1 = nn.Linear(128 * 6 * 6, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Load emotion model
model = EmotionCNN(num_classes=3)
model.load_state_dict(torch.load('emotion_cnn.pth'))
model.eval()

# Emotion classes mapping for CNN
class_indices = {'happy': 0, 'neutral': 1, 'surprise': 2}
index_to_class = {v: k for k, v in class_indices.items()}

# Emotion mapping for KNN model (happy=1, neutral=3, surprise=2)
knn_emotion_mapping = {
    'happy': 1,
    'neutral': 3,
    'surprise': 2
}

# ====================== Music Recommendation Setup ======================
# Load KNN recommendation models
try:
    scaler = joblib.load('scaler.joblib')
    knn = joblib.load('knn_model.joblib')
    df = pd.read_csv('processed_data.csv')
    features = ["danceability", "energy", "valence", "tempo"]
    print("Music recommendation models loaded successfully!")
except Exception as e:
    print(f"Error loading music models: {str(e)}")
    scaler, knn, df = None, None, None

#---------------------Helper Functions-------------------
def preprocess_image(image_data):
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    file_bytes = np.frombuffer(image_data, np.uint8)
    img = cv2.imdecode(file_bytes, cv2.IMREAD_GRAYSCALE)

    # Detect face
    faces = face_cascade.detectMultiScale(img, scaleFactor=1.1, minNeighbors=5)

    if len(faces) == 0:
        raise Exception("No face detected")

    # Crop the first face found
    (x, y, w, h) = faces[0]
    face = img[y:y+h, x:x+w]

    # Resize and normalize
    face = cv2.resize(face, (48, 48))
    face = face / 255.0
    face = np.expand_dims(face, axis=0)
    face = np.expand_dims(face, axis=0)
    img_tensor = torch.tensor(face, dtype=torch.float32)

    return img_tensor

def get_knn_recommendations(emotion):
    """Get recommendations using the trained KNN model"""
    if scaler is None or knn is None or df is None:
        return {"error": "Music recommendation models not loaded properly"}

    try:
        # Convert emotion to the KNN label (happy=1, neutral=3, surprise=2)
        if isinstance(emotion, str):
            knn_label = knn_emotion_mapping.get(emotion.lower(), 1)  # Default to happy if not found
        else:
            # If numeric, map from CNN index to KNN label
            if emotion == 0:  # CNN happy
                knn_label = 1
            elif emotion == 1:  # CNN neutral
                knn_label = 3
            elif emotion == 2:  # CNN surprise
                knn_label = 2
            else:
                knn_label = 1  # Default to happy

        # Get songs with the matching KNN label
        emotion_songs = df[df['labels'] == knn_label]

        if len(emotion_songs) == 0:
            return {"error": f"No songs found for emotion label: {knn_label}"}

        # KNN-based recommendations
        avg_features = emotion_songs[features].mean().values.reshape(1, -1)
        avg_features_scaled = scaler.transform(avg_features)
        _, indices = knn.kneighbors(avg_features_scaled)

        # Convert URIs to playable URLs and get track info
        recommendations = []
        for uri in df.iloc[indices[0]]['uri']:
            if uri.startswith('spotify:track:'):
                track_id = uri.split(':')[-1]
                try:
                    track = sp.track(uri)
                    recommendations.append({
                        'name': track['name'],
                        'artist': track['artists'][0]['name'],
                        'link': track['external_urls']['spotify'],
                        'image': track['album']['images'][0]['url'] if track['album']['images'] else None
                    })
                except:
                    recommendations.append({
                        'link': f"https://open.spotify.com/track/{track_id}",
                        'name': f"Track {track_id}",
                        'artist': "Unknown",
                        'image': None
                    })
            else:
                recommendations.append({
                    'link': uri,
                    'name': "Custom Track",
                    'artist': "Unknown",
                    'image': None
                })
        return recommendations

    except Exception as e:
        return {"error": f"Error in recommendation: {str(e)}"}

def get_spotify_track(emotion):
    """Get random track from Spotify based on emotion"""
    emotion_str = index_to_class.get(emotion, emotion) if isinstance(emotion, int) else emotion
    results = sp.search(q=f'mood:{emotion_str}', type='track', limit=10)
    tracks = results['tracks']['items']

    if not tracks:
        # Default fallback track
        track = sp.track('spotify:track:4uLU6hMCjMI75M1A2tKUQC')
        return {
            'name': track['name'],
            'artist': track['artists'][0]['name'],
            'link': track['external_urls']['spotify'],
            'image': track['album']['images'][0]['url'] if track['album']['images'] else None
        }

    track = random.choice(tracks[:5])  # Random from top 5 tracks
    return {
        'name': track['name'],
        'artist': track['artists'][0]['name'],
        'link': track['external_urls']['spotify'],
        'image': track['album']['images'][0]['url'] if track['album']['images'] else None
    }

#----------------API Endpoints--------------------
@app.route('/predict', methods=['POST'])
def predict():
    if 'file' not in request.files:
        return jsonify({'error': 'No file uploaded'}), 400

    file = request.files['file']
    if file.filename == '':
        return jsonify({'error': 'No file selected'}), 400

    try:
        # Read and preprocess the image
        image_data = file.read()
        processed_img = preprocess_image(image_data)

        # Get emotion prediction
        prediction = model(processed_img)
        predicted_class_index = torch.argmax(prediction, dim=1).item()
        predicted_class_label = index_to_class[predicted_class_index]

        # Get both KNN and Spotify recommendations
        knn_recommendations = get_knn_recommendations(predicted_class_index)  # Pass the index to handle mapping
        spotify_recommendation = get_spotify_track(predicted_class_index)

        response = {
            'predicted_class': predicted_class_label,
            'knn_recommendations': knn_recommendations if not isinstance(knn_recommendations, dict) else [],
            'spotify_recommendation': spotify_recommendation,
            'model_used': 'knn' if not (isinstance(knn_recommendations, dict) and 'error' in knn_recommendations) else 'spotify'
        }
        return jsonify(response), 200

    except Exception as e:
        return jsonify({'error': str(e)}), 500

@app.route('/recommend/<emotion>', methods=['GET'])
def recommend(emotion):
    """Direct recommendation endpoint (for testing)"""
    try:
        # Handle both string and numeric emotion inputs
        if emotion.isdigit():
            emotion_int = int(emotion)
            if emotion_int == 0:
                emotion_str = 'happy'
            elif emotion_int == 1:
                emotion_str = 'neutral'
            elif emotion_int == 2:
                emotion_str = 'surprise'
            else:
                emotion_str = 'happy'
        else:
            emotion_str = emotion.lower()
            emotion_int = class_indices.get(emotion_str, 0)

        knn_recommendations = get_knn_recommendations(emotion_int)  # Pass the index to handle mapping
        spotify_recommendation = get_spotify_track(emotion_str)

        response = {
            'requested_emotion': emotion_str,
            'knn_recommendations': knn_recommendations if not isinstance(knn_recommendations, dict) else [],
            'spotify_recommendation': spotify_recommendation,
            'model_used': 'knn' if not (isinstance(knn_recommendations, dict) and 'error' in knn_recommendations) else 'spotify'
        }
        return jsonify(response), 200
    except Exception as e:
        return jsonify({'error': str(e)}), 500

if __name__ == '__main__':
    # Start ngrok tunnel
    ngrok_tunnel = ngrok.connect(5000)
    print(' * Public URL:', ngrok_tunnel.public_url)

    # Run Flask app
    app.run(port=5000)

Music recommendation models loaded successfully!
 * Public URL: https://9cb5-34-125-205-230.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
INFO:werkzeug:127.0.0.1 - - [14/Apr/2025 01:27:40] "POST /predict HTTP/1.1" 200 -
